# ase-toolbox.ipynb
ASEによる化学シミュレーションでよく使うコードと、作ったヘルパー関数をまとめておくやつ。

### 🟡計算機を用意する(Matlantis環境)

In [ ]:
import pfp_api_client
from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from pfp_api_client.pfp.estimator import Estimator, EstimatorCalcMode

# バージョンによって計算結果が異なる場合があるため、毎回バージョンを確認する
print(f"pfp_api_client: {pfp_api_client.__version__}")

# ASECalculatorを使用するための設定
# EstimatorCalcModeは、以下のように使い分ける
# - 一般の系： EstimatorCalcMode.CRYSTAL_U0 Uなしモード
# - 酸化物など： EstimatorCalcMode.CRYSTAL　Uありモード
# - 単体有機分子： EstimatorCalcMode.MOLECULE 分子モード
estimator = Estimator(calc_mode=EstimatorCalcMode.MOLECULE)
calculator = ASECalculator(estimator)  # このcalculatorをatoms.calcに設定して使用する


### 🟡matplotlibで日本語を表示する

In [ ]:
import matplotlib.pyplot as plt

# ---matplotlibで日本語を表示する
plt.rcParams["font.family"] = [
    "DejaVu Sans",
    "Hiragino Sans",
    "Yu Gothic",
    "Meiryo",
    "Takao",
    "IPAexGothic",
    "IPAPGothic",
    "VL PGothic",
    "Noto Sans CJK JP",
]
# Windows環境での代替設定
try:
    import matplotlib.font_manager as fm

    # 日本語フォントを探す
    font_list = [
        f.name
        for f in fm.fontManager.ttflist
        if "gothic" in f.name.lower()
        or "mincho" in f.name.lower()
        or "meiryo" in f.name.lower()
    ]
    if font_list:
        plt.rcParams["font.family"] = font_list[0]
    else:
        # フォールバック: Unicode対応
        plt.rcParams["font.family"] = "DejaVu Sans"
except:
    pass

plt.rcParams["font.family"]


### 🟡可視化する

In [ ]:
from pfcc_extras.visualize.view import view_ngl
from ase.build import bulk

view_ngl(bulk("Cu"), representations=["ball+stick"], w=400, h=300)

### 🟡最適化

#### 最適化アルゴリズムについて
- 局所最適へのたどり着き方(=アルゴリズム)は、いろいろある
- 選び方
    - 基本的に`LBFGSLineSearch`か`FIRE`を使えば良いらしい
        - `LBFGSLineSearch`がうまくいかなかったら`FIRE`を使う、みたいな
    - 選ぶのが面倒なら、上の2つを組み合わせた`FIRELBFGS`を使うと良い
        - Matlantisオリジナルの最適化アルゴリズム
        - 使用方法は、2つ下のセルに示す
    - 局所最適を探すアルゴリズムの違いなだけなので、シミュレーションに大きく差が生じることはないのかもしれない
- 参考文献
    - https://docs.matlantis.com/atomistic-simulation-tutorial/ja/2_3_opt-algorithm.html

In [ ]:
from ase.optimize import FIRE, LBFGSLineSearch

opt = FIRE(atoms)
opt.run(fmax=0.001)


In [ ]:
# FIRELBFGS 最適化アルゴリズムの使い方(Matlantisオリジナル)
from matlantis_features.ase_ext.optimize import FIRELBFGS

opt = FIRELBFGS(atoms)
opt.run(fmax=0.001)


### 🟡書き出す

In [ ]:
# 内蔵ライブラリからcifファイルを作成する
from ase.build import molecule
from ase.io import write

# 出力先ディレクトリ（必要に応じて変更）
output_dir = "./"

# 分子リスト
species = ["CO2", "CO", "H2", "H2O", "O2"]

for sym in species:
    mol = molecule(sym)  # ASE 内蔵の分子ライブラリから生成
    filename = f"{output_dir}{sym}.cif"
    write(filename, mol)  # CIF 形式で保存
    print(f"Written: {filename}")


## 🔴構造を作る

### 🟡基本的な構造

In [ ]:
from ase.build import molecule, bulk, surface
from ase.io import read, write

# ---分子
co = molecule("CO")

# ---結晶
cu = bulk("Cu")
# 増やす
_cu = cu * (2, 3, 4)

# ---表面
slab = surface(
    lattice=cu, indices=(1, 1, 1), layers=2, vacuum=10.0
)*(4,4,1)  # cuの、(111)面を2層で真空層10.0Åで切り出す

# ---ファイルから
# cif,xyz,vaspからいけるらしい
# cu2o=read('Cu2O.cif')

# ---構造のコピー
# Pythonのルールとして、cu_copy = cu とするだけだと、cu_copyもcuも同じものを指し示してしまう
# (「Python 参照渡し」とかを調べるとわかる)
# copy()を使えば、別のオブジェクトとして扱える


cu_copy = cu.copy()
cu_copy[0].symbol = "O"  # コピー先の原子を変えてみる
print("=== cu_copy = cu.copy()の場合 ===")
print(f"コピー元: {cu[0].symbol}")  # コピー元: Cu
print(f"コピー先: {cu_copy[0].symbol}")  # コピー先: O
# copy()することで、コピー元とコピー先が別のものを指し示すようになる

cu_ref = cu
cu_ref[0].symbol = "O"
print("=== cu_ref = cuの場合 ===")
print(f"コピー元: {cu_ref[0].symbol}")  # コピー元: O
print(f"コピー先: {cu[0].symbol}")  # コピー先: O
# そのまま = で代入すると、コピー元とコピー先が同じものを指し示してしまうので、どちらも置き換わってしまう


### 🟡クラスター

In [ ]:
from ase.cluster import Icosahedron, Octahedron
from pfcc_extras.visualize.view import view_ngl

# ---八面体
# 正八面体
_cluster = Octahedron(
    "Cu",
    length=7,  # 層の数
    cutoff=0,  # 頂点をどれくらい切るか。0だと切らない。
)

# 切頂八面体: 頂点が切られている八面体
_cluster = Octahedron("Cu", length=7, cutoff=2)

# 正切頂八面体
_cluster = Octahedron(
    "Cu",
    length=7,  # length=3*cutoff+1で、正切頂八面体となる
    cutoff=2,
)

# 立方八面体
_cluster = Octahedron(
    "Cu",
    length=5,  # length=2*cutoff+1で、立方八面体となる
    cutoff=2,
)

# ---二十面体
_cluster = Icosahedron(
    "Cu",
    noshells=5,  # 原子の数
)

# クラスターの原子数の確認
print(f"原子数: {len(_cluster)}")

# 構造を可視化してみる
view_ngl(_cluster, representations=["ball+stick"], w=400, h=300)


### 🟡特定の原子を探す

In [ ]:
# 準備
from ase.cluster import Icosahedron
from ase.build import surface, bulk

test_slab = surface(bulk("Cu"), indices=(1, 1, 1), layers=2, vacuum=10.0) * (4, 4, 1)
test_cluster = Icosahedron("Cu", noshells=5)

#### 🟢インデックスでピンポイントに指定する

In [ ]:
from ase_toolbox.FindAtoms import find_atom_by_index

# test_slabの0番目の原子を探す
find_atom_by_index(test_slab, 0)

#### 🟢指定した原子のインデックスを調べる

In [ ]:
from ase_toolbox.FindAtoms import find_indices_by_symbol

# Cu原子のインデックスを調べる
find_indices_by_symbol(test_slab, "Cu")


#### 🟢隣接原子を探す

In [ ]:
from ase_toolbox.FindAtoms import get_neighbors

# test_slabの0番目の原子の、隣接原子を探す
neighbor_atoms = get_neighbors(test_slab, 0)

# 引数には、Atomオブジェクトを指定することもできる
neighbor_neighbor_atoms = get_neighbors(test_slab, neighbor_atoms[0])

# 返値は、インデックスのリストにすることもできる
neighbor_atoms = get_neighbors(test_slab, 0, return_type="indices")


#### 🟢(平面用)層別に分ける・層ごとのlistにする

In [ ]:
from ase_toolbox.FindAtoms import separate_layers

# test_slabを層にわける
layers = separate_layers(test_slab)

# 層の数
print(f"層の数: {len(layers)}")

# 一番下の層
layers[0]

# 一番上の層
layers[-1]


#### 🟢(クラスター用)表面・内側を探す

In [ ]:
from ase_toolbox.FindAtoms import classify_surface_atoms

# test_clusterを、表面原子と内部原子に分ける
out_atoms, in_atoms = classify_surface_atoms(test_cluster)

# 数を数える
print(f"表面原子数: {len(out_atoms)}")
print(f"内部原子数: {len(in_atoms)}")


#### 🟢重心に最も近い原子を探す

In [ ]:
from ase_toolbox.FindAtoms import separate_layers, find_central_atom

# 一番上の層を探す
layers = separate_layers(test_slab)
top_layer = layers[-1]

# top_layerの中心原子を探す
center_atom = find_central_atom(top_layer)

center_atom


#### 🟢くっつけた後の構造の中で、くっつけた原子のインデックスを知る

In [ ]:
from ase.build import add_adsorbate
from ase.build import molecule
from ase_toolbox.FindAtoms import (
    separate_layers,
    find_central_atom,
    get_appended_atom_indices,
)

# 一番上の層の真ん中を探す
layers = separate_layers(test_slab)
center_atom = find_central_atom(layers[-1])

# 真ん中に原子を追加する
mol = molecule("CO")
adsorbed_slab = test_slab.copy()
add_adsorbate(adsorbed_slab, mol, 1.6, position=center_atom)

# 新しくついた原子のインデックスを調べる
get_appended_atom_indices(before_atoms=test_slab, after_atoms=adsorbed_slab)


### 🟡構造を作るために計算する

#### 🟢配位数を計算する

In [ ]:
from ase_toolbox.FindAtoms import separate_layers, find_central_atom
from ase_toolbox.CalcValue import coordination_number

# 一番上の層の真ん中を探す
layers = separate_layers(test_slab)
center_atom = find_central_atom(layers[-1])

# 中心原子の隣接原子を探す
coordination_number(test_slab, center_atom)


#### 🟢(クラスター用)法線ベクトルを求める

In [ ]:
from ase_toolbox.CalcValue import compute_surface_normal

# test_clusterのインデックス0の原子の法線ベクトルを計算する
normal_vector = compute_surface_normal(test_cluster, 0)


### 🟡処理する

#### 🟢手動で微妙に動かす

In [ ]:
from ase_toolbox.HandleAtoms import move_atoms

# test_slabの0番目の原子を、z軸方向に1だけ移動させる
moved_slab = move_atoms(
    base_atoms=test_slab,
    target=0,
    direction=[0, 0, 1],
    distance=1,
)


#### 🟢(平面用)層を固定する

In [ ]:
from ase_toolbox.HandleAtoms import fix_layers

# test_slabを、下から2層目まで固定する
fixed_slab = fix_layers(test_slab, 2)


#### 🟢置き換える

In [ ]:
from ase_toolbox.HandleAtoms import substitute_elements
from ase_toolbox.FindAtoms import separate_layers

# test_slabの0番目、1番目、2番目の原子を、Auに置き換える
substituted_slab = substitute_elements(test_slab, [0, 1, 2], "Au")

# test_slabの一番上の層を、Cu:Au=0.8:0.2に置き換える
top_layer = separate_layers(test_slab)[-1]
substituted_slab = substitute_elements(test_slab, top_layer, {"Cu": 0.8, "Au": 0.2})


#### 🟢クラスターにくっつける

In [ ]:
from ase.build import molecule
from ase_toolbox.HandleAtoms import place_adsorbate_along_normal

# 原子を法線方向に配置する
adsorbed_cluster = place_adsorbate_along_normal(
    substrate=test_cluster,
    adsorbate=molecule("CO"),
    target_atom=0,
    distance=1.6,
)


#### 🟢表面にくっつける

In [ ]:
from ase.build import molecule
from ase_toolbox.HandleAtoms import place_adsorbate_on_surface

# 原子を法線方向に配置する
adsorbed_cluster = place_adsorbate_on_surface(
    substrate=test_slab,
    adsorbate=molecule("CO"),
    target_atom=None, # Noneで表面の中心。index|Atomで指定も可能
    height=2.4,
    position="hollow", # top, bridge, hollowが使用可能
    allow_search_surface_atom=True, # Trueの場合、target_atomが表面に無くても、そのxy座標に近い表面原子をtarget_atomにする
    inplace=False # もとの構造を置き換えるかどうか
)


### 🟡溶媒を作る

In [ ]:
# 必要に応じて、Packmolをインストールする
!git clone https://github.com/m3g/packmol.git
!cd packmol && ./configure && make


In [ ]:
from ase import Atoms
# from ase.cluster import Icosahedron
# from ase.build import bulk, surface
from ase_toolbox.BuildSolvent import ComponentSpec, build_solvated_system

# 簡単なスラブを作成
slab = Atoms("Au4", positions=[[0, 0, 0], [2, 0, 0], [0, 2, 0], [2, 2, 0]])
# slab = Icosahedron(
#     "Cu",
#     noshells=3,  # 原子の数
# )
# slab = surface(
#     lattice=bulk("Cu"), indices=(1, 1, 1), layers=2, vacuum=10.0
# )*(8,8,1)

# 溶媒成分を定義
components = [
    ComponentSpec("NaCl", 1, "Cl[Na]"),
    # ComponentSpec("ethanol", 1, "CCO"),
]

# 溶媒化システムを構築
result = build_solvated_system(
    cell=(30, 30, 40),
    structure=slab,
    components=components,
    verbose=True,
    packmol_bin="packmol/packmol",
    structure_position=(0, 0, -10),
)


## 🔴計算する

### 🟡吸着エネルギーを計算する

In [ ]:
from ase.constraints import FixAtoms
from ase.build import bulk, surface, add_adsorbate, molecule

from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from pfp_api_client.pfp.estimator import Estimator, EstimatorCalcMode

from ase_toolbox.HandleAtoms import fix_layers, separate_layers
from ase_toolbox.FindAtoms import find_central_atom
from ase_toolbox.Calculation import CAEInput, calculate_adsorption_energy

# ----------
# ---計算機の用意
# ----------
estimator = Estimator(calc_mode=EstimatorCalcMode.CRYSTAL_U0)
calc_solid = ASECalculator(estimator)  # このcalculatorをatoms.calcに設定して使用する

estimator = Estimator(calc_mode=EstimatorCalcMode.MOLECULE)
calc_molecule = ASECalculator(estimator)  # このcalculatorをatoms.calcに設定して使用する

# ----------
# ---構造の作成
# ----------
# Cu表面 (3x3x4 スラブ、下2層固定)
cu_bulk = bulk("Cu", "fcc", a=3.615)
cu_surface = surface(cu_bulk, (1, 1, 1), layers=4, vacuum=10.0) * (3, 3, 1)
cu_surface.center(axis=2)
cu_surface = fix_layers(cu_surface, 1)

# CO分子
co_molecule = molecule("CO")
co_molecule.center(axis=2)

# Cu-CO吸着構造 (top site)
cu_co_adsorbed = cu_surface.copy()
top_layer = separate_layers(cu_co_adsorbed)[-1]
center_atom = find_central_atom(top_layer)
add_adsorbate(
    cu_co_adsorbed, co_molecule, height=1.2, position=center_atom.position[:2]
)

# ----------
# ---吸着エネルギーの計算
# ----------
adsorbed_input = CAEInput(structure=cu_co_adsorbed, calc_mode="solid")

reactant_inputs = [
    CAEInput(structure=cu_surface, calc_mode="solid"),
    CAEInput(structure=co_molecule, calc_mode="solid"),
]

adsorption_energy = calculate_adsorption_energy(
    calculator_molecule=calc_molecule,
    calculator_solid=calc_solid,
    adsorbed_structure_input=adsorbed_input,
    reactant_structures_input=reactant_inputs,
    opt_fmax=0.05,
    opt_maxsteps=3000,
    enable_logging=True,
)

adsorption_energy

### 🟡生成エネルギーを計算する

In [ ]:
from ase.cluster import Icosahedron, Octahedron
from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from pfp_api_client.pfp.estimator import Estimator, EstimatorCalcMode

from ase_toolbox.HandleAtoms import substitute_elements
from ase_toolbox.Calculation import calculate_formation_energy

# ----------
# ---計算機の用意
# ----------
estimator = Estimator(calc_mode=EstimatorCalcMode.CRYSTAL_U0)
calc_solid = ASECalculator(estimator)  # このcalculatorをatoms.calcに設定して使用する

# ----------
# ---構造の作成
# ----------
cluster = Icosahedron(
    "Cu",
    noshells=5,  # 原子の数
)

cluster = substitute_elements(cluster, cluster, {"Cu": 0.5, "Au": 0.5})

# ----------
# ---生成エネルギーの計算
# ----------
formation_energy = calculate_formation_energy(calc_solid, cluster)

formation_energy


### 🟡NEBする

In [ ]:
from ase.build import molecule, fcc111
from ase.constraints import FixBondLength

from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from pfp_api_client.pfp.estimator import Estimator, EstimatorCalcMode
from matlantis_features.ase_ext.optimize import FIRELBFGS

from ase_toolbox.HandleAtoms import find_atom_by_index, add_adsorbate, move_atoms
from ase_toolbox.Calculation import run_neb, plot_energy_profile

# ---計算機を用意する
estimator = Estimator(calc_mode=EstimatorCalcMode.CRYSTAL_U0)

# ---特定のバルク構造を作る

# ---二量化前の構造
init_slab = fcc111("Cu", size=(4, 4, 4), vacuum=10.0)
atom1 = find_atom_by_index(init_slab, 53)
atom2 = find_atom_by_index(init_slab, 54)

co1 = molecule("CO")
co2 = molecule("CO")

add_adsorbate(init_slab, co1, 2.5, atom1.position)
add_adsorbate(init_slab, co2, 2.5, atom2.position)

# ---二量化後の構造
distance = 0.4
final_slab = init_slab.copy()
move_atoms(final_slab, [64, 65], (1, 0, 0), distance, inplace=True)
move_atoms(final_slab, [66, 67], (1, 0, 0), distance, inplace=True)

# ---C-C結合を強制的に維持する(そのままoptすると離れちゃうので)
constraint = FixBondLength(a1=65, a2=67)
final_slab.set_constraint(constraint)

# ---nebを実行する
images, energies = run_neb(init_slab, final_slab, 14, FIRELBFGS, estimator)

# ---描画する
fig, ax = plot_energy_profile(energies)


### 🟡ギブス自由エネルギーを計算する

In [ ]:
from ase.build import surface, molecule, bulk, add_adsorbate
from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from pfp_api_client.pfp.estimator import Estimator, EstimatorCalcMode

from ase_toolbox.HandleAtoms import separate_layers, find_central_atom, fix_layers
from ase_toolbox.Calculation import calculate_delta_g, CGFEInput


# 例: HO* + H+ + e- -> H2O + *
# ----------
# ---計算機の用意
# ----------
estimator = Estimator(calc_mode=EstimatorCalcMode.CRYSTAL_U0)
calc_solid = ASECalculator(estimator)  # このcalculatorをatoms.calcに設定して使用する

estimator = Estimator(calc_mode=EstimatorCalcMode.MOLECULE)
calc_molecule = ASECalculator(estimator)  # このcalculatorをatoms.calcに設定して使用する

# ---構造の用意

# 4x4x4のPt(111)表面を作成

# ---Pt(111)表面（4x4の表面、4層）
pt_bulk = bulk("Pt", a=3.92)  # Ptの格子定数
pt_slab = surface(pt_bulk, indices=(1, 1, 1), layers=4, vacuum=10.0)
pt_slab = pt_slab * (4, 4, 1)  # 4x4に拡張

# ---清浄なPt表面（生成物用）
clean_pt_slab = pt_slab.copy()

# ---OH分子
oh_molecule = molecule("OH")
oh_molecule.rotate(180, v="y")

# ---Pt表面にOHを吸着させた構造
# 表面の中央付近の原子を選択（top site吸着を想定）
oh_on_pt = pt_slab.copy()
layers = separate_layers(oh_on_pt)
center_atom = find_central_atom(layers[-1])

add_adsorbate(oh_on_pt, oh_molecule, height=2.0, position=center_atom.position[0:2])
oh_on_pt.center(vacuum=10.0, axis=2)

# 下を固定
oh_on_pt = fix_layers(oh_on_pt, 2)
clean_pt_slab = fix_layers(clean_pt_slab, 2)

# H2O分子
h2o_molecule = molecule("H2O")

# ---計算の実行
delta_g = calculate_delta_g(
    calculator_molecule=calc_molecule,
    calculator_solid=calc_solid,
    reactants=[
        CGFEInput(
            structure=oh_on_pt,  # Pt-OH構造
            calc_mode="HarmonicThermo",
            vibrate_indices=[64, 65, 19, 35, 51, 23, 39, 55],
        ),
        "CHE",  # H+ + e-
    ],
    products=[
        CGFEInput(
            structure=h2o_molecule,  # H2O（気相）
            calc_mode="IdealGasThermo",
            vibrate_indices=None,
            geometry="nonlinear",
            symmetry_number=2,
            spin_multiplicity=1,
        ),
        CGFEInput(
            structure=clean_pt_slab,  # 清浄なPt表面
            calc_mode="HarmonicThermo",  # 固体表面なのでHarmonicThermo
            vibrate_indices=[],  # 振動させる原子なし
        ),
    ],
    temperature=298.15,
    pressure=101325,
    electrode_potential=0.0,
    pH=0,
    opt_maxsteps=1500,
    enable_logging=True,
)

# ---結果の表示
print(f"ΔG for HO* + H+ + e- -> H2O + * = {delta_g:.3f} eV")


### 🟡格子定数を計算する

In [ ]:
from ase.build import bulk
from ase_toolbox.Calculation import optimize_lattice_constant

from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from pfp_api_client.pfp.estimator import Estimator, EstimatorCalcMode

# ---計算機の用意
estimator = Estimator(calc_mode=EstimatorCalcMode.CRYSTAL_U0)
calc_solid = ASECalculator(estimator)  # このcalculatorをatoms.calcに設定して使用する

# ---原子構造の作成
atoms = bulk("Si", "diamond", a=5.4)

# ---格子の最適化・格子定数の取得
lattice_result = optimize_lattice_constant(atoms, calc_solid)
lattice_result
